### Environment Setup

In [1]:
%pip install -qU langchain langchain-openai langchain-core python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
_ = load_dotenv()

### Interfacing with Model

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [5]:
# default model
llm.model_name

'gpt-3.5-turbo'

In [6]:
# custom model
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [9]:
# simplest invoke
out = llm.invoke("Hello, who are you?")
print(out.content)

Hello! I'm an AI language model created by OpenAI, here to help answer your questions and provide information on a wide range of topics. How can I assist you today?


In [10]:
print(f"{out.usage_metadata=}")
print(f"{out.response_metadata['model_name']=}")

out.usage_metadata={'input_tokens': 13, 'output_tokens': 36, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
out.response_metadata['model_name']='gpt-4o-2024-08-06'


### Using Message Objects

Above we invoked LLM with raw string. There are other ways to invoke the model. One of them is to use message objects. This way, we can provide more information to the model. For example, we can provide a chat history to the model. We can also provide a system prompt to the model.

1. invoke (string): Takes in a text parameter as string, internally converts it to a HumanMessage and returns a AIMessage with content
2. invoke (messages as objects): Takes a list of specific message objects, returns a message. This way, system prompt or a chat history can be given.
3. invoke (messages as strings): Takes a list of tuples indicating role and its message ('system',"Your name is Ali")

Since the response is a AIMessage object, content is read to get the message

In [13]:
# single message object (should be a list in any case)
from langchain_core.messages import HumanMessage

llm.invoke([HumanMessage("Hello, who are you?")])

AIMessage(content="Hello! I'm an AI language model created by OpenAI, here to help answer your questions and provide information on a wide range of topics. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 13, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None}, id='run-3f197fd2-4cd3-44ce-bc33-110227872516-0', usage_metadata={'input_tokens': 13, 'output_tokens': 36, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# multiple message objects including system prompt
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage(
        content="You are a helpful assistant! Your name is Bob."
    ),
    HumanMessage(
        content="What is your name?"
    )
]

llm.invoke(messages)

AIMessage(content='My name is Bob. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 27, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e437ae2-2615-4810-8834-bf481d3615cc-0', usage_metadata={'input_tokens': 27, 'output_tokens': 13, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
# easy way to create multiple role messages
llm.invoke([
    ('system',"Your name is Ali"),
    ('human',"What's your name?")
    ])

AIMessage(content='My name is Ali. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 19, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None}, id='run-b8a54aef-c8b4-45cd-b2e8-8a65dfe13fe0-0', usage_metadata={'input_tokens': 19, 'output_tokens': 13, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Prompt Templates

Templates are used to generate prompts using input variables. This templates can be passed to the model together with their input variables. 

In [7]:
# a prompt template with single message type (becomes human message) and single input variable
from langchain_core.prompts import ChatPromptTemplate

template_str = """
Rate a product score of based on the following user comment from 0 to 5 where 5 is the maximum and 0 is the minimum.
comment: {comment}
"""
template = ChatPromptTemplate.from_template(template_str)
template

ChatPromptTemplate(input_variables=['comment'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['comment'], input_types={}, partial_variables={}, template='\nRate a product score of based on the following user comment from 0 to 5 where 5 is the maximum and 0 is the minimum.\ncomment: {comment}\n'), additional_kwargs={})])

In [35]:
# filling the template and invoking llm
out = llm.invoke(template.invoke({"comment":"I hate this product. I'll throw it away. It's useless."}))
print(out.content)

Based on the comment provided, the product would likely receive a score of 0. The user expresses strong dissatisfaction and indicates that they find the product completely useless.


Above we created a template from a string. This is treated as HumanMessagePromptTemplate by default. We can also create a SystemMessagePromptTemplate and combination of them.

In [37]:
# a prompt template with multiple role message types and multiple input variables
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# create the list of messages
template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant named {assistant_name}. You introduce yourself, including your name, and reply to users' product comments"),
    HumanMessagePromptTemplate.from_template("Users comment about the product: {comment}"),
])
template

ChatPromptTemplate(input_variables=['assistant_name', 'comment'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['assistant_name'], input_types={}, partial_variables={}, template="You are a helpful assistant named {assistant_name}. You introduce yourself, including your name, and reply to users' product comments"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['comment'], input_types={}, partial_variables={}, template='Users comment about the product: {comment}'), additional_kwargs={})])

In [38]:
out = llm.invoke(template.invoke({"assistant_name":"Joe", "comment": "I'll throw it away. It's useless."}))
out.content

"Hello, I'm Joe. I'm sorry to hear that you're not satisfied with the product. Could you share a bit more about what issues you're experiencing? Maybe I can help find a solution or suggest an alternative."

In [10]:
# easy way to create a prompt template with multiple role message types and multiple input variables
template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant named {assistant_name}. You introduce yourself, including your name, and reply to users' product comments"),
    ("human","Users comment about the product: {comment}"),
])
template

ChatPromptTemplate(input_variables=['assistant_name', 'comment'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['assistant_name'], input_types={}, partial_variables={}, template="You are a helpful assistant named {assistant_name}. You introduce yourself, including your name, and reply to users' product comments"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['comment'], input_types={}, partial_variables={}, template='Users comment about the product: {comment}'), additional_kwargs={})])

In [11]:
out = llm.invoke(template.invoke({"assistant_name":"Joe", "comment": "I'll throw it away. It's useless."}))
out.content

"Hi there, I'm Joe. I'm sorry to hear that you're not satisfied with the product. Could you let me know what specifically didn't meet your expectations? Maybe I can help find a solution or suggest an alternative."

### Appendix

Above we filled the template with a dictionary and using invoke. Calling invoke creates a ChatPromptValue that can be passed to the model.
There are other ways to fill a template.

1. invoke: Parameters must be passed as dictionary. Result is same as format_prompt that can be passed directly to chat model.
2. format_messages: will create message list that can be passed directly to chat model
3. format_prompt: will create ChatPromptValue object that has the 'messages' list that can be also passed to chat model
4. format: will create a single string combining&stringifying messages that can be passed to llm model or chat model.
5. messages[i].prompt.format: will fill individual message in template

In [40]:
# invoke with dictionary resulting ChatPromptValue that can be passed to chat model.
template.invoke({"assistant_name":"Joe", "comment": "I'll throw it away. It's useless."})

ChatPromptValue(messages=[SystemMessage(content="You are a helpful assistant named Joe. You introduce yourself, including your name, and reply to users' product comments", additional_kwargs={}, response_metadata={}), HumanMessage(content="Users comment about the product: I'll throw it away. It's useless.", additional_kwargs={}, response_metadata={})])

In [42]:
# format_messages with named arguments resulting message list that can be passed to chat model.
template.format_messages(assistant_name="Joe", comment="I'll throw it away. It's useless.")

[SystemMessage(content="You are a helpful assistant named Joe. You introduce yourself, including your name, and reply to users' product comments", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Users comment about the product: I'll throw it away. It's useless.", additional_kwargs={}, response_metadata={})]

In [45]:
# format_prompt with named arguments resulting ChatPromptValue that can be passed to chat model.
template.format_prompt(assistant_name="Joe", comment="I'll throw it away. It's useless.")

ChatPromptValue(messages=[SystemMessage(content="You are a helpful assistant named Joe. You introduce yourself, including your name, and reply to users' product comments", additional_kwargs={}, response_metadata={}), HumanMessage(content="Users comment about the product: I'll throw it away. It's useless.", additional_kwargs={}, response_metadata={})])

In [46]:
# format with named arguments resulting raw string, but not usable.
template.format(assistant_name="Joe", comment="I'll throw it away. It's useless.")

"System: You are a helpful assistant named Joe. You introduce yourself, including your name, and reply to users' product comments\nHuman: Users comment about the product: I'll throw it away. It's useless."

In [51]:
# we can format single message in the template
template.messages[0].prompt.format(assistant_name="Bob")

"You are a helpful assistant named Bob. You introduce yourself, including your name, and reply to users' product comments"